# boids

See:

* Craig Reynolds' page [Boids](http://www.red3d.com/cwr/boids/)


* Conrad Parker's page [Boids]
https://vergenet.net/~conrad/boids/pseudocode.html


In [49]:
class Point2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # Subtract two points to get a vector start_point --> self.
    def __sub__(self, start_point):
        return Vector2d(self.x - start_point.x, self.y - start_point.y)

    # Add this point to a vector to get a new point
    # (Or add the coordinates of two points, e.g. to calculate an average.)
    def __add__(self, other):
        return Point2d(self.x + other.x, self.y + other.y)

    # Scaling isn't really a point operation.
    # It's used by this program to calculate averages.
    # Scale this point by multiplying.
    def __mul__(self, scale):
        return Point2d(self.x * scale, self.y * scale)

    # Scale this point by dividing.
    def __truediv__(self, scale):
        return self * (1.0 / scale)

    def __str__(self):
        return f'({self.x}, {self.y})'

    def draw(self, canvas, color):
        RADIUS = 3
        canvas.create_oval(
            self.x - RADIUS, self.y - RADIUS,
            self.x + RADIUS, self.y + RADIUS,
            fill=color, outline=color)

In [50]:
import math

class Vector2d:
    # Initialize from either (a, b) coordinates or two points a --> b.
    def __init__(self, a, b):
        if type(a) is Point2d:
            # Initialize from two points a --> b.
            self.x = b.x - a.x
            self.y = b.y - a.y
        else:
            # Initialize from (a, b) coordinates.
            self.x = a
            self.y = b

    # Add two vectors to get a new vector or
    # add the vector to a point to get a new point.
    def __add__(self, other):
        if type(other) is Vector2d:
            # Add two vectors to get a new vector.
            return Vector2d(self.x + other.x, self.y + other.y)
        else:
            # Add the vector to a point to get a new point.
            return Point2d(self.x + other.x, self.y + other.y)

    # Return the negation of this vector.
    def __neg__(self):
        return Vector2d(-self.x, -self.y)

    # Subtract two vectors.
    def __sub__(self, other):
        return self + -other

    # Scale this vector by multiplying.
    def __mul__(self, scale):
        return Vector2d(self.x * scale, self.y * scale)

    # Scale this vector by dividing.
    def __truediv__(self, scale):
        return self * (1.0 / scale)

    # Scale the vector.
    # Return self so we can use it in further calculations.
    def scale(self, scale):
        self.x *= scale
        self.y *= scale
        return self

    # Return the vector's length.
    def length(self):
        return math.sqrt(self.x * self.x + self.y * self.y)

    # Set the vector's length to new_length.
    # Return self so we can use it in further calculations.
    def set_length(self, new_length):
        old_length = self.length()
        if old_length < 0.01: return self # Don't divide by zero.
        self.x *= new_length / old_length
        self.y *= new_length / old_length
        return self

    # Set the vector's length to 1.
    # Return self so we can use it in further calculations.
    def normalize(self):
        return self.set_length(1)

    def __str__(self):
        return f'<{self.x}, {self.y}>'

    def draw(self, canvas, start_point, color):
        end_point = self + start_point
        canvas.create_line(
            start_point.x, start_point.y,
            end_point.x, end_point.y,
            fill=color)

In [51]:
class Boid:
    def __init__(self, canvas, radius, color, location, velocity):
        # Save parameters.
        self.canvas = canvas
        self.radius = radius
        self.location = location
        self.velocity = velocity

        # Make the boid's circle.
        self.circle = self.canvas.create_oval(
            self.location.x - self.radius, self.location.y - self.radius,
            self.location.x + self.radius, self.location.y + self.radius,
            fill=color, outline=color)

    def __str__(self):
        return f'Boid({self.location}, {self.velocity})'

    # Return a vector self --> other.
    def vector_to(self, other):
        return Vector2d(other.location.x - self.location.x, other.location.y - self.location.y)

    # Return the distance between this boid and another one.
    def distance(self, other):
        return math.sqrt((other.location.x - self.location.x) ** 2 + (other.location.y - self.location.y) ** 2)

    # Update the Boid's velocity and position.
    def update(self, boids, elapsed, cohesion_wgt, separation_wgt,
            alignment_wgt, target_wgt, max_speed, neighbor_distance, target):
        #For rule 1
        center_point = Point2d(0, 0)
        cohesion_vector = Vector2d(0, 0)
        #For rule 2
        separation_vector = Vector2d(0, 0)
        #For rule 3
        alignment_vector = Vector2d(0, 0)
        for boid in boids:
            if boid is not self:
                center_point += boid.location
                if (boid.location - self.location).length() <= neighbor_distance:
                    separation_vector -= (boid.location - self.location)
                alignment_vector += boid.velocity
        if len(boids) > 1: #don't divide by zero
            center_point /= (len(boids) - 1)
            cohesion_vector = Vector2d(self.location, center_point)      
            alignment_vector /= (len(boids) - 1)
        alignment_vector = (alignment_vector - self.velocity)
        target_vector = Vector2d(self.location, target.location)
        self.new_velocity = self.velocity + \
            cohesion_vector * cohesion_wgt + \
            separation_vector * separation_wgt + \
            alignment_vector * alignment_wgt + \
            target_vector * target_wgt
        if self.new_velocity.length() > max_speed:
            self.new_velocity.set_length(max_speed)

    # Move the boid to its new location.
    def move(self, elapsed):
        self.velocity = self.new_velocity
        self.location += self.velocity * elapsed
        self.move_circle()

    # Move the boid's circle to its current position.
    def move_circle(self):
        self.canvas.moveto(self.circle,
            self.location.x - self.radius,
            self.location.y - self.radius)

In [52]:
import tkinter as tk

# Get the text in an Entry widget and
# convert it to an int.
def get_int(entry):
    return int(entry.get())

# Get the text in an Entry widget and
# convert it to a float.
def get_float(entry):
    return float(entry.get())

# Make Label and Entry widgets for a field.
# Return the Entry widget.
def make_field(parent, label_width, label_text, entry_width, entry_default):
    frame = tk.Frame(parent)
    frame.pack(side=tk.TOP)

    label = tk.Label(frame, text=label_text, width=label_width, anchor=tk.W)
    label.pack(side=tk.LEFT)

    entry = tk.Entry(frame, width=entry_width, justify='right')
    entry.insert(tk.END, entry_default)
    entry.pack(side=tk.LEFT)

    return entry

In [53]:
import tkinter as tk
from timeit import default_timer as timer
import random

# Geometry constants.
MARGIN = 5
WINDOW_WID = 500
WINDOW_HGT = 300
CANVAS_HGT = WINDOW_HGT - 2 * MARGIN
CANVAS_WID = CANVAS_HGT
TARGET_RADIUS = 4
TARGET_COLOR = 'blue'
BOID_RADIUS = 2
BOID_COLOR = 'red'

TICK_MS = 50

class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.running = False

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('boids')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Build the rest of the UI.
        self.build_ui()

        # We have no boids yet.
        self.boids = []

        # Put the target in the middle of the canvas.
        self.target = Boid(self.canvas, TARGET_RADIUS, TARGET_COLOR,
            Point2d(CANVAS_WID / 2, CANVAS_HGT / 2), Vector2d(0, 0))

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    def build_ui(self):
        # Make the drawing canvas.
        self.canvas = tk.Canvas(self.window, bg='white',
            borderwidth=1, highlightthickness=0, width=CANVAS_WID, height=CANVAS_HGT)
        self.canvas.pack(side=tk.LEFT, padx=MARGIN, pady=MARGIN, anchor=tk.NW)
        self.canvas.bind('<Motion>', self.track_mouse)

        # Make a frame to hold labels, text boxes, and buttons.
        right_frame = tk.Frame(self.window)
        right_frame.pack(side=tk.LEFT, padx=MARGIN, pady=MARGIN, anchor=tk.NW)

        # Separation, alignment, cohesion, and target.
        LABEL_WID = 18
        self.separation_entry = make_field(right_frame, LABEL_WID, 'Separation Weight:', 4, 10)
        self.alignment_entry = make_field(right_frame, LABEL_WID, 'Alignment Weight:', 4, 1)
        self.cohesion_entry = make_field(right_frame, LABEL_WID, 'Cohesion Weight:', 4, 1)
        self.target_entry = make_field(right_frame, LABEL_WID, 'Target Weight:', 4, 5)

        # Make a vertical gap.
        gap = tk.Frame(right_frame, width=1, height=10)
        gap.pack(side=tk.TOP)

        # Max speed, neighbor distance, and # boids.
        self.max_speed_entry = make_field(right_frame, LABEL_WID, 'Max Speed:', 4, 100)
        self.neighbor_distance_entry = make_field(right_frame, LABEL_WID, 'Neighborhood Radius:', 4, 10)
        self.num_boids_entry = make_field(right_frame, LABEL_WID, '# Boids:', 4, 20)

        # Make a vertical gap.
        gap = tk.Frame(right_frame, width=1, height=10)
        gap.pack(side=tk.TOP)

        # Run button.
        self.run_button = tk.Button(right_frame, text='Run', width=7, command=self.run)
        self.run_button.pack(side=tk.TOP)

    # Move the target.
    def track_mouse(self, event):
        self.target.location = Point2d(event.x, event.y)
        self.target.move_circle()

    def run(self):
        if self.running:
            # Stop running.
            self.running = False
            self.run_button.config(text='Run')
        else:
            # Start running.
            self.running = True
            self.run_button.config(text='Stop')

            # Get parameters.
            self.separation_wgt = get_float(self.separation_entry)
            self.alignment_wgt = get_float(self.alignment_entry)
            self.cohesion_wgt = get_float(self.cohesion_entry)
            self.target_wgt = get_float(self.target_entry)
            self.max_speed = get_float(self.max_speed_entry)
            self.neighbor_distance = get_float(self.neighbor_distance_entry)
            self.num_boids = get_int(self.num_boids_entry)

            # Make some random boids.
            self.reset_boids()
            self.boids = []
            for i in range(self.num_boids):
                location = self.target.location + Vector2d(
                    random.randint(-20, 20),
                    random.randint(-20, 20)
                )
                self.boids.append(Boid(self.canvas, BOID_RADIUS, BOID_COLOR,
                    location, Vector2d(0, 0)))

            # Go!
            self.last_time = timer()
            self.window.after(TICK_MS, self.tick)

    # Update and move the boids.
    def tick(self):
        # Get the elapsed time in seconds.
        now = timer()
        elapsed = now - self.last_time
        self.last_time = now

        # Update the boids.
        for boid in self.boids:
            boid.update(self.boids, elapsed,
                self.cohesion_wgt,
                self.separation_wgt,
                self.alignment_wgt,
                self.target_wgt,
                self.max_speed,
                self.neighbor_distance,
                self.target)

        # Move the boids.
        for boid in self.boids:
            boid.move(elapsed)

        # If we're still running, schedule another tick.
        if self.running:
            self.window.after(TICK_MS, self.tick)

    # Destroy any existing boids and their circles.
    def reset_boids(self):
        for boid in self.boids:
            self.canvas.delete(boid.circle)
        self.boids = []

    def kill_callback(self):
        self.window.destroy()

In [54]:
App()